<a href="https://colab.research.google.com/github/mickeykim70/keras_study/blob/main/8_2_cats_dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. Google Drive 연결
from google.colab import drive
drive.mount('/content/drive')

# 2. Kaggle API 설정
!pip install kaggle
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

Mounted at /content/drive
